In [1]:
# STEP 1: INSTALL LIBRARIES (uncomment if running on fresh environment)
!pip install -q sentence-transformers faiss-cpu pypdf fpdf gradio tqdm

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 50.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 19.0 MB/s eta 0:00:00


In [2]:
# STEP 2: IMPORTS & CONFIG
import os
from pathlib import Path
import json
import gc
import numpy as np
from tqdm import tqdm
import faiss
from pypdf import PdfReader
from fpdf import FPDF
from sentence_transformers import SentenceTransformer
import gradio as gr
from datetime import datetime
import pandas as pd

# --- Configuration (adjust if needed) ---
DATA_ROOT = Path('/content/support_bot_data')
DATA_ROOT.mkdir(parents=True, exist_ok=True)
PDF_PATH = DATA_ROOT / 'sample_faq.pdf'
INDEX_FOLDER = DATA_ROOT / 'faiss_index'
INDEX_FOLDER.mkdir(parents=True, exist_ok=True)
INDEX_PATH = INDEX_FOLDER / 'index.faiss'
METADATA_PATH = INDEX_FOLDER / 'metadata.jsonl'
ID_COUNTER_PATH = INDEX_FOLDER / 'next_id.txt'
TRANSCRIPTS_PATH = DATA_ROOT / 'transcripts.csv'

# Embedding model config
LOCAL_EMBED_MODEL = 'all-MiniLM-L6-v2'
BATCH_SIZE = 32
embedder = SentenceTransformer(LOCAL_EMBED_MODEL)

print('Configuration ready.')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Configuration ready.


In [3]:
# STEP 3: HELPER FUNCTIONS (PDF Creation, Chunking, ID Management)

def create_faq_pdf(path=PDF_PATH):
    """Generates the sample FAQ PDF with NotoSans."""
    if path.exists():
        print('ℹ️ PDF already exists.')
        return

    font_path_str = '/content/NotoSans-Regular.ttf'
    font_url = 'https://github.com/googlefonts/noto-fonts/raw/main/hinted/ttf/NotoSans/NotoSans-Regular.ttf'
    font_path = Path(font_path_str)

    if not font_path.exists():
        print("Downloading font for PDF...")
        os.system(f"wget -q '{font_url}' -O '{font_path_str}'")
        print("Font downloaded.")

    faq_text = '''Sample Support FAQ

Q1. How do I reset my password?
Go to Settings > Account > Reset Password...

Q2. What should I do if I forget my username?
Your username is your registered email address...

Q3. How do I update my billing details?
Navigate to Settings > Billing > Update Details...

Q4. Can I cancel my subscription?
Yes. Go to Settings > Subscription > Cancel...

Q5. How do I contact support?
Email us at support@example.com...

Q6. What is the refund policy?
Refunds are available if cancellation occurs within 7 days...

Q7. Can I export my data?
Yes, under Settings > Data > Export...

Q8. Is two-factor authentication supported?
Yes, you can enable 2FA in Settings > Security.'''

    pdf = FPDF()
    pdf.add_page()
    pdf.add_font('NotoSans', '', font_path_str, uni=True)
    pdf.set_font('NotoSans', size=12)
    pdf.multi_cell(0, 8, faq_text)
    pdf.output(str(path))
    print('Created sample FAQ PDF.')

def chunk_text(text, max_chars=800, overlap=150):
    """Splits text into overlapping chunks based on character count."""
    text = text.replace('\r\n', '\n').strip()
    if not text:
        return []
    if len(text) <= max_chars:
        return [text]
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + max_chars, len(text))
        chunks.append(text[start:end].strip())
        start = end - overlap
        if start < 0:
            start = 0
        if start >= len(text):
            break
    return chunks

def load_next_id():
    """Loads the next available unique ID for a vector."""
    if ID_COUNTER_PATH.exists():
        return int(ID_COUNTER_PATH.read_text().strip())
    return 0

def save_next_id(n):
    """Saves the next available unique ID."""
    ID_COUNTER_PATH.write_text(str(int(n)))

def load_all_metadata():
    """Loads all metadata from the .jsonl file."""
    if not METADATA_PATH.exists():
        return []
    meta = []
    with open(METADATA_PATH, 'r', encoding='utf-8') as f:
        for line in f:
            if line.strip():
                meta.append(json.loads(line))
    return meta

In [5]:
# STEP 4: INGESTION

def ingest_pdf_lowmem(pdf_path, source_name='kb', batch_size=BATCH_SIZE):
    pdf_path = Path(pdf_path)
    print(f'Reading PDF pages from {pdf_path}...')
    reader = PdfReader(str(pdf_path))
    pages_texts = [p.extract_text() for p in reader.pages if p.extract_text()]
    full_text = '\n'.join(pages_texts).strip()
    if not full_text:
        raise ValueError('No text extracted from PDF.')

    #Smarter chunking:
    import re
    # Try FAQ-style split: Q1., Q2., etc.
    faq_chunks = re.split(r'(?=Q\d+\.)', full_text)
    faq_chunks = [c.strip() for c in faq_chunks if c.strip()]

    if len(faq_chunks) > 1:
        chunks = faq_chunks
        print(f"Detected FAQ format → split into {len(chunks)} Q&A chunks")
    else:
        # Fallback: split paragraphs by double newlines
        initial_chunks = [chunk for chunk in full_text.split('\n\n') if chunk.strip()]
        final_chunks = []
        for paragraph in initial_chunks:
            if len(paragraph) > 1000:
                final_chunks.extend(chunk_text(paragraph, max_chars=800, overlap=150))
            else:
                final_chunks.append(paragraph)
        chunks = final_chunks
        print(f"Paragraph/character split → {len(chunks)} chunks")

    # Continue as before...
    if INDEX_PATH.exists():
        index = faiss.read_index(str(INDEX_PATH))
    else:
        sample_emb = embedder.encode([chunks[0]])
        dim = sample_emb.shape[1]
        base_index = faiss.IndexFlatIP(dim)
        index = faiss.IndexIDMap(base_index)

    next_id = load_next_id()
    with open(METADATA_PATH, 'a', encoding='utf-8') as meta_file:
        for start in tqdm(range(0, len(chunks), batch_size), desc='Indexing batches'):
            end = min(start + batch_size, len(chunks))
            batch_texts = chunks[start:end]
            batch_embs = embedder.encode(batch_texts, show_progress_bar=False, convert_to_numpy=True).astype('float32')
            faiss.normalize_L2(batch_embs)

            ids = np.arange(next_id, next_id + len(batch_texts)).astype('int64')
            index.add_with_ids(batch_embs, ids)

            for i, txt in enumerate(batch_texts):
                rec = {'assigned_id': int(ids[i]), 'source': source_name, 'text': txt}
                meta_file.write(json.dumps(rec) + '\n')

            next_id += len(batch_texts)
            gc.collect()

    print('Saving index and next available ID...')
    faiss.write_index(index, str(INDEX_PATH))
    save_next_id(next_id)
    print(f'Done. Total vectors in index: {index.ntotal}')
    return index


In [6]:
# STEP 5: INITIAL DATA PREPARATION
create_faq_pdf()
if not INDEX_PATH.exists():
    print("Performing initial ingestion of sample PDF...")
    index = ingest_pdf_lowmem(str(PDF_PATH), source_name='sample_faq')
else:
    print("Loading existing index...")
    index = faiss.read_index(str(INDEX_PATH))

metadata_list = load_all_metadata()
print(f'Pipeline ready. Index has {index.ntotal} vectors. Metadata has {len(metadata_list)} records.')

Font downloaded.
Created sample FAQ PDF.
Performing initial ingestion of sample PDF...
Reading PDF pages from /content/support_bot_data/sample_faq.pdf...
Detected FAQ format → split into 9 Q&A chunks


Indexing batches: 100%|██████████| 1/1 [00:00<00:00,  1.17it/s]

Saving index and next available ID...
Done. Total vectors in index: 9
Pipeline ready. Index has 9 vectors. Metadata has 9 records.


In [7]:
# STEP 6: RETRIEVAL + ANSWERING + TRANSCRIPTS

def retrieve_topk_and_map(index, metadata, query, k=4):
    """Retrieves top k results and maps them to their metadata."""
    q_emb = embedder.encode([query], convert_to_numpy=True).astype('float32')
    faiss.normalize_L2(q_emb)
    distances, ids = index.search(q_emb, k)

    meta_by_id = {m['assigned_id']: m for m in metadata}
    retrieved = []

    for sim, doc_id in zip(distances[0], ids[0]):
        if doc_id != -1 and doc_id in meta_by_id:
            retrieved.append((meta_by_id[doc_id], float(sim)))

    return retrieved

def save_transcript(timestamp, session_id, role, message, sources=""):
    """Helper function to save transcript entries."""
    if not TRANSCRIPTS_PATH.exists():
        pd.DataFrame(columns=['timestamp','session_id','role','message','sources']).to_csv(TRANSCRIPTS_PATH, index=False)
    pd.DataFrame([{'timestamp': timestamp, 'session_id': session_id, 'role': role, 'message': message, 'sources': sources}])\
      .to_csv(TRANSCRIPTS_PATH, mode='a', header=False, index=False)

# Heuristic follow-up generator (no external LLM)
def generate_followups_from_snippets(retrieved, max_followups=4):
    """
    Create simple follow-up questions by turning the leading part of each snippet into
    short prompts like "Tell me more about ...", "How do I ...", etc.
    'retrieved' is list of (meta, sim) tuples.
    """
    followups = []
    for rec, sim in retrieved:
        text = rec.get('text', '').strip()
        if not text:
            continue
        first_sentence = text.split('\n')[0].split('. ')[0]
        words = first_sentence.split()
        phrase = ' '.join(words[:10])
        phrase = phrase.strip().rstrip(',:;')  # cleanup
        if phrase:
            lowered = phrase.lower()
            if lowered.startswith(('how', 'what', 'when', 'where', 'why', 'can', 'is', 'are', 'do', 'does')):
                q = phrase + '?'
            else:
                q = f"Tell me about {phrase}?"
            if len(q) > 120:
                q = q[:117].rstrip() + "?"
            followups.append(q)
        if len(followups) >= max_followups:
            break
    generic = [
        "Can you give an example?",
        "What are the next steps?",
        "How does this affect billing?",
        "Who can I contact for more help?"
    ]
    i = 0
    while len(followups) < max_followups and i < len(generic):
        if generic[i] not in followups:
            followups.append(generic[i])
        i += 1
    return followups[:max_followups]

def answer_with_followups(user_query, index, metadata, top_k=4, sim_threshold=0.35):
    """
    Retrieve top_k, compute a simple answer (top chunk), compute confidence,
    and produce follow-up questions.
    Returns: answer_text, sources_text, followups_list, confidence_float
    """
    retrieved = retrieve_topk_and_map(index, metadata, user_query, k=top_k)
    if not retrieved:
        return ("No relevant information found. Please escalate to human support.",
                "",
                ["Try a different query."],
                0.0)

    # compute average similarity as confidence heuristic
    sims = [sim for (_, sim) in retrieved]
    avg_sim = float(np.mean(sims)) if sims else 0.0
    # normalize similarity to 0-1 (safeguard mapping)
    conf = max(0.0, min(1.0, (avg_sim + 1.0) / 2.0))

    top_sim = sims[0] if sims else 0.0
    if top_sim < sim_threshold:
        return ("I'm not confident about the answer. Please rephrase or escalate to human support.",
                "",
                ["Rephrase the question", "Ask about account settings", "Ask about billing", "Escalate to human support"],
                conf)

    top_rec = retrieved[0][0]
    answer_text = top_rec.get('text', '').strip()

    # build sources text (shortified)
    sources = []
    for rec, sim in retrieved:
        s = f"Similarity: {sim:.3f}\nSource: {rec.get('source','unknown')}\n\n{rec.get('text','')[:600]}..."
        sources.append(s)
    sources_text = "\n\n---\n\n".join(sources)

    followups = generate_followups_from_snippets(retrieved, max_followups=4)

    return answer_text, sources_text, followups, conf

In [8]:
# STEP 7: FILE UPLOAD HANDLER (index new pdfs)
def handle_file_upload(file):
    """Handle PDF upload and indexing with robust file handling."""
    if file is None:
        return "Please upload a PDF file first."
    try:
        temp_file_path = file.name
        upload_path = DATA_ROOT / Path(temp_file_path).name

        # save uploaded file to disk
        with open(temp_file_path, "rb") as src, open(upload_path, "wb") as dst:
            dst.write(src.read())

        global index, metadata_list
        index = ingest_pdf_lowmem(upload_path, source_name=upload_path.stem)
        metadata_list = load_all_metadata()

        return f" Successfully indexed {upload_path.name}! You can now ask questions about it."
    except Exception as e:
        return f" Error processing file: {str(e)}"

In [9]:
# STEP 8: VERIFICATION TESTS (quick)
try:
    test_queries = [
        "How do I reset my password?",
        "What is the refund policy?",
        "How do I contact support?"
    ]
    print("\n Running quick verification tests...")
    for query in test_queries:
        answer, sources, followups, conf = answer_with_followups(query, index, metadata_list)
        print(f"\nQ: {query}")
        print(f"A (snippet): {answer[:120]}...")
        print(f"Found sources: {len(sources) > 0}, Confidence: {int(conf*100)}%")
    print("\n Verification tests completed!")
except Exception as e:
    print(f"\n Verification test failed: {e}")
# ------------------ Option A: chat_fn_with_followups wrapper ------------------
def chat_fn_with_followups(message, session_id='demo_user'):
    """
    Wrapper expected by the UI:
      returns (answer_str, sources_str, followups_list, confidence_float)

    Uses your existing answer_with_followups(...) function and handles logging + type safety.
    Paste this BEFORE STEP 9 (Gradio UI).
    """
    ts = datetime.utcnow().isoformat()
    # Save the user message to transcripts if possible
    try:
        save_transcript(ts, session_id, 'user', message)
    except Exception:
        # don't break if transcript saving fails
        pass

    try:
        # Call the main pipeline you already implemented
        answer, sources, followups, conf = answer_with_followups(message, index, metadata_list, top_k=4, sim_threshold=0.35)

        # Normalize types
        answer = "" if answer is None else str(answer)
        sources = "" if sources is None else str(sources)

        # Ensure followups is a list of strings
        if not isinstance(followups, (list, tuple)):
            # If it's empty or a single string, convert to list
            try:
                followups = list(followups) if hasattr(followups, '__iter__') and not isinstance(followups, (str, bytes)) else [str(followups)]
            except Exception:
                followups = [str(followups)]
        followups = [str(x) for x in followups]

        # Ensure confidence is float between 0.0 and 1.0
        try:
            conf = float(conf)
            if conf < 0.0: conf = 0.0
            if conf > 1.0: conf = 1.0
        except Exception:
            conf = 0.0

        # Save assistant reply to transcripts
        try:
            save_transcript(ts, session_id, 'assistant', answer, sources)
        except Exception:
            pass

        return answer, sources, followups, conf

    except Exception as e:
        # Log the exception and return safe defaults for the UI
        try:
            log_exception(e)
        except Exception:
            # if logging also fails, at least print
            import traceback as _tb
            print(_tb.format_exc())

        # safe defaults: user-visible friendly message + hide followups
        return ("An internal error occurred.", f"Error details logged to {LOG_PATH}", [], 0.0)
# ------------------ end wrapper ------------------


 Running quick verification tests...

Q: How do I reset my password?
A (snippet): Q1. How do I reset my password?
Go to Settings > Account > Reset Password......
Found sources: True, Confidence: 71%

Q: What is the refund policy?
A (snippet): Q6. What is the refund policy?
Refunds are available if cancellation occurs within 7 days......
Found sources: True, Confidence: 67%

Q: How do I contact support?
A (snippet): Q5. How do I contact support?
Email us at support@example.com......
Found sources: True, Confidence: 68%

 Verification tests completed!


In [10]:
# ------------------ STEP 9: GRADIO UI (safe version) ------------------

import traceback
LOG_PATH = Path('/content/support_bot_data/gradio_handler_errors.log')

def log_exception(e: Exception):
    tb = traceback.format_exc()
    print(tb)  # also prints to notebook cell
    try:
        with open(LOG_PATH, 'a', encoding='utf-8') as f:
            f.write(f"\n\n[{datetime.utcnow().isoformat()}] Exception:\n")
            f.write(tb)
    except Exception:
        pass

def safe_btn_updates_from_list(followups, max_buttons=4):
    updates = []
    for i in range(max_buttons):
        try:
            label = str(followups[i]) if i < len(followups) else ""
        except Exception:
            label = ""
        visible = bool(label)
        updates.append(gr.update(value=label, visible=visible))
    return updates

def on_send_safe(q, sid):
    try:
        if not q:
            followups = ["Try: How do I reset my password?", "What is the refund policy?", "Who can I contact for support?", "How to export data?"]
            btns = safe_btn_updates_from_list(followups)
            return ("Please type a question.", "", "Confidence: 0%", btns[0], btns[1], btns[2], btns[3])

        ans, src, followups, conf = chat_fn_with_followups(q, sid)
        conf_text = f"Confidence: {int(conf*100)}%"
        btns = safe_btn_updates_from_list(followups)
        return (str(ans), str(src), conf_text, btns[0], btns[1], btns[2], btns[3])
    except Exception as e:
        log_exception(e)
        hidden_btns = [gr.update(value="", visible=False) for _ in range(4)]
        return ("An internal error occurred.", f"Error details logged to {LOG_PATH}", "Confidence: 0%", hidden_btns[0], hidden_btns[1], hidden_btns[2], hidden_btns[3])

def run_followup_and_ask_safe(label, sid):
    try:
        if not label:
            hidden_btns = [gr.update(value="", visible=False) for _ in range(4)]
            return ("", "", "Confidence: 0%", hidden_btns[0], hidden_btns[1], hidden_btns[2], hidden_btns[3])
        return on_send_safe(label, sid)
    except Exception as e:
        log_exception(e)
        hidden_btns = [gr.update(value="", visible=False) for _ in range(4)]
        return ("An internal error occurred.", f"Error details logged to {LOG_PATH}", "Confidence: 0%", hidden_btns[0], hidden_btns[1], hidden_btns[2], hidden_btns[3])

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# Support Chatbot RAG\nAsk questions about the FAQ or upload your own PDF documentation.")

    with gr.Row():
        with gr.Column(scale=2):
            answer_box = gr.Textbox(label="Answer", lines=8, interactive=False)
            sources_box = gr.Textbox(label="Retrieved Sources", lines=10, interactive=False)
            confidence_box = gr.Textbox(label="Confidence", lines=1, interactive=False, value="Confidence: 0%")
            gr.Markdown("### Suggested follow-up questions (click to ask):")
            fbtn1 = gr.Button("", visible=False)
            fbtn2 = gr.Button("", visible=False)
            fbtn3 = gr.Button("", visible=False)
            fbtn4 = gr.Button("", visible=False)

        with gr.Column(scale=1):
            gr.Markdown("### Upload Documentation")
            file_upload = gr.File(label="PDF File", file_types=[".pdf"])
            upload_btn = gr.Button("Index PDF", variant="secondary")
            upload_status = gr.Textbox(label="Upload Status", interactive=False)

            gr.Markdown("### Ask a Question")
            user_input = gr.Textbox(label="Your question", placeholder="Ask about the documentation...")
            send_btn = gr.Button("Send", variant="primary")
            session_id = gr.Textbox(label="Session ID", value="demo_user", visible=False)

    upload_btn.click(handle_file_upload, inputs=[file_upload], outputs=[upload_status])

    send_btn.click(
        on_send_safe,
        inputs=[user_input, session_id],
        outputs=[answer_box, sources_box, confidence_box, fbtn1, fbtn2, fbtn3, fbtn4]
    )

    fbtn1.click(run_followup_and_ask_safe, inputs=[fbtn1, session_id],
                outputs=[answer_box, sources_box, confidence_box, fbtn1, fbtn2, fbtn3, fbtn4])
    fbtn2.click(run_followup_and_ask_safe, inputs=[fbtn2, session_id],
                outputs=[answer_box, sources_box, confidence_box, fbtn1, fbtn2, fbtn3, fbtn4])
    fbtn3.click(run_followup_and_ask_safe, inputs=[fbtn3, session_id],
                outputs=[answer_box, sources_box, confidence_box, fbtn1, fbtn2, fbtn3, fbtn4])
    fbtn4.click(run_followup_and_ask_safe, inputs=[fbtn4, session_id],
                outputs=[answer_box, sources_box, confidence_box, fbtn1, fbtn2, fbtn3, fbtn4])

print("\n Launching Gradio UI...")
demo.launch(share=True, debug=False)

!tail -n 40 /content/support_bot_data/gradio_handler_errors.log

# are we in the same kernel and is the wrapper defined?
print("chat_fn_with_followups" in globals())
import inspect
print("wrapper source available? ->", "chat_fn_with_followups" in globals() and inspect.getsource(chat_fn_with_followups)[:120])

res = chat_fn_with_followups("How do I reset my password?", session_id="demo_user")
print("Returned tuple length:", len(res))
print("Types:", [type(x) for x in res])
print("\nAnswer (first 400 chars):\n", res[0][:400])
print("\nSources (first 400 chars):\n", res[1][:400])
print("\nFollowups:", res[2])
print("Confidence:", res[3])

print("Index total vectors:", getattr(index, "ntotal", None))
print("Metadata records:", len(metadata_list))
# Optionally print first metadata item
if len(metadata_list) > 0:
    import json
    print("First metadata record:", json.dumps(metadata_list[0], indent=2)[:800])

import shutil
shutil.rmtree(INDEX_FOLDER)   # delete old FAISS index + metadata
INDEX_FOLDER.mkdir(parents=True, exist_ok=True)

print("Index total vectors:", index.ntotal)
print("Metadata records:", len(metadata_list))

res = chat_fn_with_followups("What is the refund policy?", session_id="demo_user")
print(res[0])   # should return just Q6’s answer

res = chat_fn_with_followups("What is the refund policy?", session_id="demo_user")
print("nswer:\n", res[0])
print("\nSources:\n", res[1])
print("\nFollowups:", res[2])
print("Confidence:", res[3])


 Launching Gradio UI...
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://27945cb048cb2219be.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


tail: cannot open '/content/support_bot_data/gradio_handler_errors.log' for reading: No such file or directory
True
wrapper source available? -> def chat_fn_with_followups(message, session_id='demo_user'):
    """
    Wrapper expected by the UI:
      returns (answ
Returned tuple length: 4
Types: [<class 'str'>, <class 'str'>, <class 'list'>, <class 'float'>]

Answer (first 400 chars):
 Q1. How do I reset my password?
Go to Settings > Account > Reset Password...

Sources (first 400 chars):
 Similarity: 0.799
Source: sample_faq

Q1. How do I reset my password?
Go to Settings > Account > Reset Password......

---

Similarity: 0.348
Source: sample_faq

Q2. What should I do if I forget my username?
Your username is your registered email address......

---

Similarity: 0.285
Source: sample_faq

Q5. How do I contact support?
Email us at support@example.com......

---

Similarity: 0.279
Sou

Followups: ['Tell me about Q1?', 'Tell me about Q2?', 'Tell me about Q5?', 'Tell me about Q3?']
Confid

/tmp/ipython-input-1442546365.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat()
/tmp/ipython-input-1442546365.py:26: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().isoformat()


nswer:
 Q6. What is the refund policy?
Refunds are available if cancellation occurs within 7 days...

Sources:
 Similarity: 0.770
Source: sample_faq

Q6. What is the refund policy?
Refunds are available if cancellation occurs within 7 days......

---

Similarity: 0.243
Source: sample_faq

Q5. How do I contact support?
Email us at support@example.com......

---

Similarity: 0.225
Source: sample_faq

Q4. Can I cancel my subscription?
Yes. Go to Settings > Subscription > Cancel......

---

Similarity: 0.173
Source: sample_faq

Q3. How do I update my billing details?
Navigate to Settings > Billing > Update Details......

Followups: ['Tell me about Q6?', 'Tell me about Q5?', 'Tell me about Q4?', 'Tell me about Q3?']
Confidence: 0.6765284892171621
